In [3]:
import numpy as np
import matplotlib
import pandas as pd
import tensorflow as tf

dataset = pd.read_csv('/Users/wzhang/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data (1).csv')

labels = dataset.iloc[:,16:].values

vals = pd.DataFrame(dataset.iloc[:,4:15].values)

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(vals,labels,test_size=0.2,random_state=0)
X_train = np.array(X_train)


In [1]:
from keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization, GRU, LayerNormalization, MultiHeadAttention,TimeDistributed, Input, Flatten
from sklearn.ensemble import RandomForestRegressor

2024-06-27 15:45:57.651737: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_GRU():
    model = Sequential()

    model.add(GRU(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(GRU(units=100, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(GRU(units=100))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008), loss='mae')
    model.fit(
        X_train, Y_train, validation_split = 0.2, batch_size=24, epochs=2000
    )
    return model

In [7]:
from sklearn.linear_model import Lasso
def create_lasso():
    model = Lasso(alpha=100)
    model.fit(X_train,Y_train)
    return model


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
def create_LSTM():

    # Define the LSTM model
    model = Sequential()

    model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=100))
    model.add(Dropout(0.2))

    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')


    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00075), loss='mae')
    # Fit the model to the training data
    history = model.fit(X_train, Y_train, epochs=1000, batch_size=64, validation_data=(X_test, Y_test), shuffle=False)
    return model


In [6]:
def create_mlp():
    # Initialize MLP model
    mlp_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression task
    ])

    # Compile the model
    mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss='mae')

    # Print model summary
    mlp_model.summary()

    # Train the model
    history = mlp_model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)

    # Evaluate model performance
    return mlp_model


In [4]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model




In [5]:
from sklearn.linear_model import Ridge
def create_ridge():
    model = Ridge(alpha=100)
    model.fit(X_train,Y_train)
    return model

In [8]:
from sklearn.svm import SVR
def create_SVM():
    # Initialize SVM regressor
    svm_model = SVR(kernel='rbf', C=10000, gamma='scale')

    # Train the model
    svm_model.fit(X_train, Y_train)
    return svm_model


In [10]:
# gru = create_GRU()
# lasso = create_lasso()
# lstm = create_LSTM()
mlp = create_mlp()
rf = create_rf()
# ridge = create_ridge()
# svm = create_SVM()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,393 (13.25 KB)

 Trainable params: 3,393 (13.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 23.3004 - val_loss: 23.6844
Epoch 2/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 23.9553 - val_loss: 20.7076
Epoch 3/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 21.5882 - val_loss: 18.7870
Epoch 4/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17.4006 - val_loss: 17.2153
Epoch 5/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17.5907 - val_loss: 16.5404
Epoch 6/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15.8747 - val_loss: 15.9586
Epoch 7/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.0193 - val_loss: 15.0291
Epoch 8/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.3345 - val_loss: 16.0471
Epoch 9/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.4646 - val_loss: 14.5076
Epoch 10/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.4450 - val_loss: 14.2712
Epoch 11/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 15.0694 - val_loss: 13.8443
Epoch 12/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [11]:
import deepforest
from deepforest import CascadeForestRegressor

In [12]:
from tensorflow.keras.layers import concatenate


# gruPredict = gru.predict(X_train)
# lassoPredict = lasso.predict(X_train)
# lstmPredict = lstm.predict(X_train)
mlpPredict = mlp.predict(X_train)
rfPredict = rf.predict(X_train)
# ridgePredict = ridge.predict(X_train)
# svmPredict = svm.predict(X_train)
# lassoPredict.resize(671,1)
rfPredict.resize(671,1)
# svmPredict.resize(671,1)
trainData = concatenate([mlpPredict,rfPredict])

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [37]:
model = CascadeForestRegressor(n_estimators = 50, criterion='squared_error', backend='sklearn')

In [13]:
import numpy as np

In [14]:
np.__version__

'1.23.1'

In [38]:
model.fit(trainData, Y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2024-06-27 20:55:52.918] Start to fit the model:
[2024-06-27 20:55:52.918] Fitting cascade layer = 0 
[2024-06-27 20:56:10.815] layer = 0  | Val MSE = 138.89660 | Elapsed = 17.897 s
[2024-06-27 20:56:10.883] Fitting cascade layer = 1 
[2024-06-27 20:59:05.750] layer = 1  | Val MSE = 159.88770 | Elapsed = 174.867 s
[2024-06-27 20:59:05.750] Early stopping counter: 1 out of 2
[2024-06-27 20:59:05.806] Fitting cascade layer = 2 
[2024-06-27 21:02:03.061] layer = 2  | Val MSE = 185.95459 | Elapsed = 177.255 s
[2024-06-27 21:02:03.061] Early stopping counter: 2 out of 2
[2024-06-27 21:02:03.061] Handling early stopping
[2024-06-27 21:02:03.108] The optimal number of layers: 1


In [17]:
# gruPredictTest = gru.predict(X_test)
# lassoPredictTest = lasso.predict(X_test)
# lstmPredictTest = lstm.predict(X_test)
mlpPredictTest = mlp.predict(X_test)
rfPredictTest = rf.predict(X_test)
# ridgePredictTest = ridge.predict(X_test)
# svmPredictTest = svm.predict(X_test)
# svmPredictTest.resize(168,1)
rfPredictTest.resize(168,1)
# lassoPredictTest.resize(168,1)
testData = concatenate([mlpPredictTest, rfPredictTest])

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 874us/step


In [19]:
res = model.predict(testData)
print(res)

[2024-06-27 20:47:31.649] Start to evalute the model:
[2024-06-27 20:47:31.649] Evaluating cascade layer = 0 
[[  4.538048  ]
 [589.640689  ]
 [  6.648925  ]
 [ 53.351529  ]
 [  7.48610017]
 [ 87.3450575 ]
 [ 25.214371  ]
 [ 59.266652  ]
 [ 12.669522  ]
 [  7.3129855 ]
 [ 44.606091  ]
 [174.31206   ]
 [  2.824626  ]
 [ 15.31132   ]
 [  5.592785  ]
 [  9.729625  ]
 [  3.541738  ]
 [  3.575182  ]
 [  8.34600667]
 [ 80.834858  ]
 [ 39.34796   ]
 [  2.015637  ]
 [  8.38674183]
 [  3.067232  ]
 [ 15.4129078 ]
 [  9.49822817]
 [ 29.368231  ]
 [  9.11499067]
 [104.766778  ]
 [  7.98100967]
 [ 14.9861015 ]
 [  3.894204  ]
 [ 31.422151  ]
 [ 45.972941  ]
 [ 42.198516  ]
 [ 11.14393   ]
 [ 17.712453  ]
 [  8.25065217]
 [ 12.365369  ]
 [ 14.9432598 ]
 [  8.35776567]
 [ 16.9925275 ]
 [ 10.4872275 ]
 [  2.0706    ]
 [  2.236232  ]
 [  6.905646  ]
 [ 21.223399  ]
 [ 14.5554038 ]
 [  3.509147  ]
 [  1.5956    ]
 [ 24.1190105 ]
 [ 15.680696  ]
 [162.622817  ]
 [ 63.257189  ]
 [ 34.948722  ]
 [ 17.1532

In [20]:
from sklearn.metrics import r2_score 
r2 = r2_score(Y_test, res) 
print(r2)

0.9266260875324814


In [21]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test,res)
print(mae)

7.546833614285715


In [22]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(Y_test,res)
print(mape)

0.5425587782530025


In [23]:
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(Y_test,res)
print(rmse)

16.566458703067685


In [25]:
import shap
print(testData.numpy())
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(testData.numpy())



[[5.91814852e+00 4.68060017e+00]
 [5.77580811e+02 3.90060699e+02]
 [7.16818762e+00 6.85659981e+00]
 [4.41382942e+01 5.15993996e+01]
 [1.35139828e+01 8.21889973e+00]
 [8.66902466e+01 8.74632034e+01]
 [1.49783106e+01 2.96245995e+01]
 [1.00221451e+02 5.94417992e+01]
 [1.16826782e+01 1.23114004e+01]
 [9.42167568e+00 7.58599997e+00]
 [4.18660774e+01 4.50588989e+01]
 [1.16405144e+02 1.50219406e+02]
 [3.98707461e+00 3.30480003e+00]
 [1.73633232e+01 1.53299999e+01]
 [5.99115896e+00 5.94189978e+00]
 [8.68227291e+00 1.04138002e+01]
 [5.99274778e+00 3.48350000e+00]
 [4.39484167e+00 3.62089992e+00]
 [9.15020752e+00 8.11540031e+00]
 [7.54608078e+01 8.35129013e+01]
 [3.40919952e+01 3.95985985e+01]
 [5.37567091e+00 2.74219990e+00]
 [1.08808985e+01 9.38809967e+00]
 [5.38325644e+00 3.40709996e+00]
 [1.63550568e+01 1.52896996e+01]
 [9.46097088e+00 8.86800003e+00]
 [1.80455971e+01 3.10678997e+01]
 [1.19295206e+01 9.86719990e+00]
 [2.06244385e+02 8.63820038e+01]
 [7.13566828e+00 8.21770000e+00]
 [1.127394

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: CascadeForestRegressor(n_estimators=50, random_state=1)

In [39]:
print(model.get_layer_feature_importances(0))

[0.40095557 0.59904443]
